
# The Path to Chess Masterhood

## Motivation 

As an expansion of the first chess web scraping project ("Elite Chess Grandmaster Web Scraping Project"), the goal for this project is to produce another large data set that combines player attributes with rating history. This time, we are interested in chess players rated 2200-2847. 


## Introduction 

Along with chess ratings comes the possibility of being titled. Chess players that have successfully reached 2200 usually receive the title of chess master. The goal for this project is to try to get a dataset that represents the rating progression of various chess master titles. See the following links for information about ratings:

- https://en.wikipedia.org/wiki/Elo_rating_system 
- https://en.wikipedia.org/wiki/Chess_title 



## Limitations 

It is important to mention that some chess titles can be achieved under a rating of 2200. Additionally, not all players above 2200 choose to acquire their titles. Moreover, chess masters that have already earned their titles can dip under the 2200 threshold. The reason this 2200 cutoff was chosen is because of the sheer size of the data set. As chess players get weaker the amount of chess players drastically increases. To illustrate using the attribute data set used in this project, there are only 949 chess players above 2500. If we include chess players 2200 and above the number skyrockets to 20,763. As we shall see in this project, extracting each recorded rating for these 20,700 chess players quickly produces an extremely large data table with over 3 million rows. 

Therefore, for the sake of keeping this data set manageable, we are only going to extract chess players that represent "master strength." Because the web scraping method I am using takes over 7 hours to extract the rating histories of 20,000 chess players from [fide.com](https://ratings.fide.com/), I am choosing to extract the chess players in 4 separate batches based on rating intervals. Trying to separate the players by titles would take too much time to verify. Because some chess players are banned or are under investigation, they need to be filtered out so the web scraping functions can perform their tasks. Verifying chess player status by rating intervals proved to be the most efficient method. 




# Data Acquisition 

Similar to the first web scraping project, we need to web-scrape rating history data from the [world chess federation website](https://ratings.fide.com/) and combine it with the attribute data that we get from downloading the May chess player rating list. 



In [1]:
# Load libraries 
import pandas as pd
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
from requests_html import HTMLSession
from pprint import pprint

In [21]:
# Read in FIDE player characteristics (sex, federation, etc)
file = 'players_list_foa_may.txt'
chess = pd.read_fwf(file)

Because trying to filter for all chess players above 2200 creates a ridiculously large data frame that creates memory errors later on. The solution I came up with was to split the table into smaller more manageable rating intervals. My plan is to eventually merge all the tables together at the very end.

In [3]:
#Filter for chess players across various rating ranges
chess_players_2200s =  chess[(chess["SRtng"] >= 2200 ) &  (chess["SRtng"] < 2300 ) ]
chess_players_2300s =  chess[(chess["SRtng"] >= 2300 ) &  (chess["SRtng"] < 2400 ) ]
chess_players_2400s =  chess[(chess["SRtng"] >= 2400 ) &  (chess["SRtng"] < 2500 ) ]
chess_players_2500s_plus =  chess[chess["SRtng"] >= 2500]

This time instead of web scraping URLs directly from the top chess player rating list, we wil be using the ID Numbers of chess players to create URLs that will directly take us to chess player rating histories. 


In [4]:
#Function that excludes problematic chess players 
def exclude_players(data):
    data_clean = data[(data["ID Number"] != 11600098) & (data["ID Number"] != 14108836) & (data["ID Number"] != 14106329) & (data["ID Number"] != 929662) & (data["ID Number"] != 901202)]
    return(data_clean)
## Function that creates list of chess player urls using chess player ids  
def create_prepped_urls(data):
    player_ids = list(data["ID Number"])
    player_ids_strings= [str(ids) for ids in player_ids]
    player_urls_prepped = ['https://ratings.fide.com/profile/' + id + '/chart' for id in player_ids_strings]
    return(player_urls_prepped)



In [5]:
# Prepare the 2200 chess player urls 
chess_players_2200s_clean = exclude_players(chess_players_2200s)
chess_players_2200s_urls = create_prepped_urls(chess_players_2200s_clean)

# Prepare the 2300 chess player urls 
chess_players_2300s_clean = exclude_players(chess_players_2300s)
chess_players_2300s_urls = create_prepped_urls(chess_players_2300s_clean)

# Prepare the 2400 chess player urls 
chess_players_2400s_clean = exclude_players(chess_players_2400s)
chess_players_2400s_urls = create_prepped_urls(chess_players_2400s_clean)

# Prepare the 2500 chess player and above urls 
chess_players_2500s_plus_clean = exclude_players(chess_players_2500s_plus)
chess_players_2500s_plus_clean_urls = create_prepped_urls(chess_players_2500s_plus_clean)

In [6]:
# Trim function used to eventually set FIDE ids as the key for the tables so that I can merge history and characteristics data
def trim_url(x):
    y = int(x.replace("https://ratings.fide.com/profile/", "").replace('/chart',""))
    return(y)



The two next blocks were used to store player rating histories in a dictionary. I created a function that web scrapes each chess player’s FIDE webpage for their rating progression and uses id as the key for each table nested in the dictionary.


In [7]:
# Function that extracts chess player rating history tables from fide website and stores them in dictionary using player ids as the keys
def extract_player_rating_history(links):
    player_stats = {}
    for link in links: 
        id=trim_url(link)
        page = requests.get(link).text
        dfs = pd.read_html(page, attrs = {'class' : 'profile-table profile-table_chart-table'})
        player_stats[id] = dfs[0]
    return(player_stats)




For transparency, this next block is how I found specific player pages that could not be scraped (because they were banned, under investigation, or had an error with their ID on FIDE's website). I checked each rating interval using this function (the tedious process will not be shown). This next block performs the same task as the extraction function above except it returns the problematic chess players' webpages.


In [ ]:
'''
player_stats = {}
for link in all_player_urls_prepped: 
    id=trim_url(link)
    page = requests.get(link).text
    try: 
        dfs = pd.read_html(page, attrs = {'class' : 'profile-table profile-table_chart-table'})
    except ValueError:
        print("Error:" + link )
'''




In [8]:
#Perform rating history extraction for 2200s
chess_player_2200s_history = extract_player_rating_history(chess_players_2200s_urls)

In [10]:

#Perform rating history extraction for 2300s
chess_player_2300s_history = extract_player_rating_history(chess_players_2300s_urls)

In [11]:

#Perform rating history extraction for 2400s
chess_player_2400s_history = extract_player_rating_history(chess_players_2400s_urls)

In [12]:

#Perform rating history extraction for 2500 and above
chess_player_2500s_plus_history = extract_player_rating_history(chess_players_2500s_plus_clean_urls)


The next step is to create an ID Number column using the ID keys that were assigned to each nested table in the dictionaries because we want to eventually perform a merge of these rating tables and the attributes from the first data frames.

In [13]:
# Function that creates an ID Number Column for each table and sets all values in that column  to the key of the table
def attach_id(chess_player_2500s_history):
    tags = chess_player_2500s_history.keys()
    for tag in tags: 
        chess_player_2500s_history[tag]["ID Number"] = tag
    return chess_player_2500s_history



In [14]:
#Perform id attachement for 2200s
chess_player_2200s_history_with_id = attach_id(chess_player_2200s_history)

#Perform id attachement for 2300s
chess_player_2300s_history_with_id = attach_id(chess_player_2300s_history)

#Perform id attachement for 2400s
chess_player_2400s_history_with_id = attach_id(chess_player_2400s_history)

#Perform id attachement for 2500s and above 
chess_player_2500s_plus_history_with_id = attach_id(chess_player_2500s_plus_history)

Now let's free the tables from the dictionaries.

In [15]:
#Fuction for turning dictionary into list and then turning list into a massive dataframe
def convert_dict_to_df(players_history): 
    list_of_player_dfs= list(players_history.values())
    all_player_stats_table= list_of_player_dfs[0].append(list_of_player_dfs[1:-1])
    return(all_player_stats_table)



In [16]:
# Perform conversion for 2200s 
player_stat_table_2200s= convert_dict_to_df(chess_player_2200s_history_with_id)

# Perform conversion for 2300s 
player_stat_table_2300s= convert_dict_to_df(chess_player_2300s_history_with_id)

# Perform conversion for 2400s 
player_stat_table_2400s= convert_dict_to_df(chess_player_2400s_history_with_id)

# Perform conversion for 2500s and above
player_stat_table_2500s_plus= convert_dict_to_df(chess_player_2500s_plus_history_with_id)



Part of what made this project difficult was the amount of memory that was required for each of these tables. The next few blocks significantly reduce the amount of memory by cutting out unnecessary columns from the rating history and attribute tables. 

In [17]:
player_stat_table_2500s_plus_cut = player_stat_table_2500s_plus[["Period","ID Number", "RTNG","RAPID RTNG", "BLITZ RTNG"]]
chess_players_2500s_plus_attributes_clean= chess_players_2500s_plus_clean[['ID Number','Tit', 'Name', 'Fed', 'Sex', 'B-day']]

player_stat_table_2400s_cut = player_stat_table_2400s[["Period","ID Number", "RTNG","RAPID RTNG", "BLITZ RTNG"]]
chess_players_2400s_attributes_clean= chess_players_2400s_clean[['ID Number', 'Name','Tit', 'Fed', 'Sex', 'B-day']]


player_stat_table_2300s_cut = player_stat_table_2300s[["Period","ID Number", "RTNG","RAPID RTNG", "BLITZ RTNG"]]
chess_players_2300s_attributes_clean= chess_players_2300s_clean[['ID Number', 'Name','Tit', 'Fed', 'Sex', 'B-day']]

player_stat_table_2200s_cut = player_stat_table_2200s[["Period","ID Number", "RTNG","RAPID RTNG", "BLITZ RTNG"]]
chess_players_2200s_attributes_clean= chess_players_2200s_clean[['ID Number', 'Name','Tit', 'Fed', 'Sex', 'B-day']]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12038 entries, 410 to 998884
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID Number  12038 non-null  int64 
 1   Name       12038 non-null  object
 2   Tit        4802 non-null   object
 3   Fed        12038 non-null  object
 4   Sex        12038 non-null  object
 5   B-day      12038 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 470.2+ KB
None


Here come the final merges and filters to create our final tables. 

In [18]:
#Function that merges attribute data from above with the rating history data 
def merging_tables(attributes, history):
    masters_table = attributes.merge(history, on= "ID Number", how= "outer")
#Filter to get rid of single chess player records 
    masters_table_filtered = masters_table[masters_table["ID Number"].map(masters_table["ID Number"].value_counts()) > 1]
    return(masters_table_filtered)


In [19]:
# Perform final merge and filter for 2200s
masters_table_filtered_2200s= merging_tables(chess_players_2200s_attributes_clean, player_stat_table_2200s_cut)
print(masters_table_filtered_2200s.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1792173 entries, 0 to 1792173
Data columns (total 10 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ID Number   int64  
 1   Name        object 
 2   Tit         object 
 3   Fed         object 
 4   Sex         object 
 5   B-day       int64  
 6   Period      object 
 7   RTNG        object 
 8   RAPID RTNG  float64
 9   BLITZ RTNG  float64
dtypes: float64(2), int64(2), object(6)
memory usage: 109.4+ MB
None


In [21]:
# Perform final merge and filter for 2300s
masters_table_filtered_2300s= merging_tables(chess_players_2300s_attributes_clean, player_stat_table_2300s_cut)
print(masters_table_filtered_2300s.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 827419 entries, 0 to 827418
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID Number   827419 non-null  int64  
 1   Name        827419 non-null  object 
 2   Tit         652041 non-null  object 
 3   Fed         827419 non-null  object 
 4   Sex         827419 non-null  object 
 5   B-day       827419 non-null  int64  
 6   Period      827419 non-null  object 
 7   RTNG        825716 non-null  float64
 8   RAPID RTNG  226458 non-null  float64
 9   BLITZ RTNG  223783 non-null  float64
dtypes: float64(3), int64(2), object(5)
memory usage: 53.7+ MB
None


In [22]:
# Perform final merge and filter for 2400s
masters_table_filtered_2400s= merging_tables(chess_players_2400s_attributes_clean, player_stat_table_2400s_cut)
print(masters_table_filtered_2400s.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 327970 entries, 0 to 327969
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID Number   327970 non-null  int64  
 1   Name        327970 non-null  object 
 2   Tit         309229 non-null  object 
 3   Fed         327970 non-null  object 
 4   Sex         327970 non-null  object 
 5   B-day       327970 non-null  int64  
 6   Period      327970 non-null  object 
 7   RTNG        327269 non-null  float64
 8   RAPID RTNG  122842 non-null  float64
 9   BLITZ RTNG  125498 non-null  float64
dtypes: float64(3), int64(2), object(5)
memory usage: 21.3+ MB
None


In [23]:

# Perform final merge and filter for 2500s
masters_table_filtered_2500s= merging_tables(chess_players_2500s_plus_attributes_clean, player_stat_table_2500s_plus_cut)
print(masters_table_filtered_2500s.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 145669 entries, 0 to 145668
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID Number   145669 non-null  int64  
 1   Tit         144894 non-null  object 
 2   Name        145669 non-null  object 
 3   Fed         145669 non-null  object 
 4   Sex         145669 non-null  object 
 5   B-day       145669 non-null  int64  
 6   Period      145669 non-null  object 
 7   RTNG        145565 non-null  float64
 8   RAPID RTNG  75478 non-null   float64
 9   BLITZ RTNG  77358 non-null   float64
dtypes: float64(3), int64(2), object(5)
memory usage: 9.4+ MB
None


In [24]:
all_chess_masters_table = pd.concat([masters_table_filtered_2200s, masters_table_filtered_2300s,masters_table_filtered_2400s, masters_table_filtered_2500s],ignore_index= True)
print(all_chess_masters_table.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3093231 entries, 0 to 3093230
Data columns (total 10 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ID Number   int64  
 1   Name        object 
 2   Tit         object 
 3   Fed         object 
 4   Sex         object 
 5   B-day       int64  
 6   Period      object 
 7   RTNG        object 
 8   RAPID RTNG  float64
 9   BLITZ RTNG  float64
dtypes: float64(2), int64(2), object(6)
memory usage: 165.2+ MB
None


In [25]:
#Save final table as csv
all_chess_masters_table.to_csv("all_chess_masters.csv")


# Sources 

For chess player ratings for the month of may check out: 
https://ratings.fide.com/download_lists.phtml